In [1]:
# Imports
import torch
from dask_jobqueue import SLURMCluster
from distributed import Client
from causalflows_complexity import run_experiment

In [2]:
seeds = 50
nsamples = 1000

In [3]:
cluster = SLURMCluster(
    n_workers=0,
    memory="20GB",
    processes=1,
    cores=1,
    scheduler_options={
        "dashboard_address": ":10092",
        "allowed_failures": 10
    },
    job_cpu=1,
    walltime="24:0:0",
)
cluster.adapt(minimum=0, maximum=200)
client = Client(cluster)

/nfs/ghome/live/danceh/.local/mambaforge/envs/causalflow/lib/python3.11/site-packages/distributed/node.py:187: UserWarning: Port 10092 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 43839 instead
  warnings.warn(


In [4]:
futures = []
for seed in range(seeds):
    f1 = client.submit(run_experiment, seed, N = nsamples, noise_dist = "normal")
    f2 = client.submit(run_experiment, seed, N = nsamples, noise_dist = "gamma")
    f3 = client.submit(run_experiment, seed, N = nsamples, noise_dist = "cauchy")
    f4 = client.submit(run_experiment, seed, N = nsamples, noise_dist = "inversegamma")
    f5 = client.submit(run_experiment, seed, N = nsamples, noise_dist = "rademacher")
    futures += [f1,f2,f3,f4,f5]

In [5]:
futures

[<Future: pending, key: run_experiment-1b692569bff4016ecd41dca3836a0dfa>,
 <Future: pending, key: run_experiment-3f603c78e6103d38749d60c0bb09e941>,
 <Future: pending, key: run_experiment-a67b8bbce103d1938b41baf01b6853c7>,
 <Future: pending, key: run_experiment-2cb474210c1ed3a2529cbca71296b645>,
 <Future: pending, key: run_experiment-2c43e8e1e43a05432baf3b2d8d36c4af>,
 <Future: pending, key: run_experiment-9b6d4d07016edc418b72452085ff60b0>,
 <Future: pending, key: run_experiment-14a2174b61640e8f75ac36b6ecfa80cb>,
 <Future: pending, key: run_experiment-e8a23929d70c1565024fec05a2045104>,
 <Future: pending, key: run_experiment-424f26242b5490a35cf7d0890e380b7e>,
 <Future: pending, key: run_experiment-d7b558cd0a362f0b021a41b664acf539>,
 <Future: pending, key: run_experiment-1d04cef991e4300b9d16249d0d7c7e7d>,
 <Future: pending, key: run_experiment-9b9a3869479f993ca62513dc81b4f94d>,
 <Future: pending, key: run_experiment-392c95e9df5e8495a6c088209cd8ae32>,
 <Future: pending, key: run_experiment

In [6]:
results = client.gather(futures)

In [7]:
client.close()
cluster.close()

In [8]:
torch.save(f = "causalflow_complexity_results_n={0}_trials={1}.pt".format(nsamples, seeds), obj = results)